In [11]:
import os
import json
import numpy as np

In [10]:
def processAndFilterInGameData(inputFilePath, outputDirectory):
    with open(inputFilePath, 'r', encoding='utf-8') as file:
        ingameData = json.load(file)

    # Initialize structure to hold filtered data, including frameInterval
    relevantData = {
        'frameInterval': ingameData['info'].get('frameInterval', 60000),
        'frames': []
    }

    if 'frames' not in ingameData['info']:
        print(f"Skipping {inputFilePath}, 'frames' key not found.")
        return

    for frame in ingameData['info']['frames']:
        # Create a new frame structure to be added to relevantData
        newFrame = {
            'participantFrames': {},
            'events': []
        }

        # Process participant frames for IDs 3 and 8
        for participantId, participantInfo in frame['participantFrames'].items():
            if int(participantId) in [3, 8]:
                newFrame['participantFrames'][participantId] = participantInfo
        
        # Process events based on your criteria (if needed, you can add here)
        for event in frame['events']:
            # Example criteria: include events of certain types involving participants 3 or 8
            if event['type'] in ['CHAMPION_KILL', 'ELITE_MONSTER_KILL', 'BUILDING_KILL']:
                participants = [event.get('killerId')] + event.get('assistingParticipantIds', [])
                if 'victimId' in event:
                    participants.append(event['victimId'])

                if any(pid in [3, 8] for pid in participants if pid is not None):
                    newFrame['events'].append(event)
        
        # Add the newFrame to relevantData if it contains filtered participantFrames or events
        if newFrame['participantFrames'] or newFrame['events']:
            relevantData['frames'].append(newFrame)

    # Ensure the output directory exists
    if not os.path.exists(outputDirectory):
        os.makedirs(outputDirectory)

    outputFilePath = os.path.join(outputDirectory, "filtered_" + os.path.basename(inputFilePath))
    with open(outputFilePath, 'w', encoding='utf-8') as outFile:
        json.dump(relevantData, outFile, indent=4)
    print(f"Processed and saved filtered data to {outputFilePath}")

# Example usage
inputFilePath = "../DataProcessing/TestData/cleanedGameData.json"  # Update to the correct path
outputDirectory = "../DataProcessing/FilteredData"  # Update to the desired output directory
processAndFilterInGameData(inputFilePath, outputDirectory)


Processed and saved filtered data to ../DataProcessing/FilteredData\filtered_cleanedGameData.json


In [16]:
def consolidateAndFilterFirst50Games(directoryPath, outputFilePath):
    consolidatedData = {}
    gamesProcessed = 0

    for filename in sorted(os.listdir(directoryPath)):
        if gamesProcessed >= 50:
            break  # Stop after processing the first 50 games

        filePath = os.path.join(directoryPath, filename)
        
        # Skip directories or non-JSON files
        if os.path.isdir(filePath) or not filePath.endswith('.json'):
            continue

        with open(filePath, 'r', encoding='utf-8') as file:
            ingameData = json.load(file)
        
        # Initialize the structure for filtered game data
        relevantData = {
            'frameInterval': ingameData['info'].get('frameInterval', 60000),
            'frames': [],
            'gameEnd': None  # Placeholder for GAME_END event data
        }

        for frame in ingameData['info'].get('frames', []):
            newFrame = {
                'participantFrames': {},
                'events': []
            }
            
            # Process participant frames for IDs 3 and 8
            for participantId, participantInfo in frame.get('participantFrames', {}).items():
                if int(participantId) in [3, 8]:  # Adjust IDs as necessary
                    newFrame['participantFrames'][participantId] = participantInfo

            # Process events based on your criteria
            for event in frame.get('events', []):
                # Include CHAMPION_KILL, ELITE_MONSTER_KILL, BUILDING_KILL events
                if event['type'] in ['CHAMPION_KILL', 'ELITE_MONSTER_KILL', 'BUILDING_KILL']:
                    participants = [event.get('killerId')] + event.get('assistingParticipantIds', [])
                    if 'victimId' in event:
                        participants.append(event['victimId'])
                    if any(pid in [3, 8] for pid in participants if pid is not None):
                        newFrame['events'].append(event)

                # Look for GAME_END event to capture winning team and timestamp
                elif event['type'] == 'GAME_END':
                    relevantData['gameEnd'] = {
                        'winningTeam': event.get('winningTeam'),
                        'timestamp': frame.get('timestamp')
                    }
                    break  # Assuming only one GAME_END event per game

            if newFrame['participantFrames'] or newFrame['events']:
                relevantData['frames'].append(newFrame)
        
        # Use matchId as key to include filtered game data
        matchId = ingameData.get('metadata', {}).get('matchId', f'unknown_{gamesProcessed}')
        consolidatedData[matchId] = relevantData
        gamesProcessed += 1

    # Ensure the output directory exists
    outputDirectory = os.path.dirname(outputFilePath)
    if not os.path.exists(outputDirectory):
        os.makedirs(outputDirectory)

    with open(outputFilePath, 'w', encoding='utf-8') as outFile:
        json.dump(consolidatedData, outFile, indent=4)
    print(f"Consolidated and filtered data for the first 50 games has been saved to {outputFilePath}")


In [17]:
directoryPath = "D:\EUW\IngameData\FinalCleanedIronMatchJsons"
outputFilePath = "D:\EUW\IngameData\IronMidIngameDataTest.json"
consolidateAndFilterFirst50Games(directoryPath, outputFilePath)

Consolidated and filtered data for the first 50 games has been saved to D:\EUW\IngameData\IronMidIngameDataTest.json
